In [1]:
from custom.datasets import Meiosis_Dataset
from custom.models import Net_Period_Classification
from custom.utils import accuracy, weight_balance, random_sample
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import ConcatDataset, DataLoader
from torch import optim
import numpy as np
from itertools import chain

In [2]:
# dataset = Meiosis_Dataset(path='../SC_figures', label={'leptotene', 'zygotene', 'pachytene_ab', 'pachytene_n'}, target=[0,0,0,0])

In [3]:
dataset_1 = Meiosis_Dataset(path='../SC_figures', label='zygotene', target=0)
indicies_1, indicies_2 = random_sample(len(dataset_1), .9)
dataset_1_t = Meiosis_Dataset(path='../SC_figures', label='zygotene', target=0)
dataset_1_t.im_paths = [dataset_1_t.im_paths[index] for index in indicies_1]
dataset_1_v = Meiosis_Dataset(path='../SC_figures', label='zygotene', target=0)
dataset_1_v.im_paths = [dataset_1_v.im_paths[index] for index in indicies_2]

dataset_2 = Meiosis_Dataset(path='../SC_figures', label='pachytene_ab', target=1)
indicies_1, indicies_2 = random_sample(len(dataset_2), .9)
dataset_2_t = Meiosis_Dataset(path='../SC_figures', label='pachytene_ab', target=1)
dataset_2_t.im_paths = [dataset_2_t.im_paths[index] for index in indicies_1]
dataset_2_v = Meiosis_Dataset(path='../SC_figures', label='pachytene_ab', target=1)
dataset_2_v.im_paths = [dataset_2_v.im_paths[index] for index in indicies_2]

dataset_3 = Meiosis_Dataset(path='../SC_figures', label='pachytene_n', target=1)
indicies_1, indicies_2 = random_sample(len(dataset_3), .9)
dataset_3_t = Meiosis_Dataset(path='../SC_figures', label='pachytene_n', target=1)
dataset_3_t.im_paths = [dataset_3_t.im_paths[index] for index in indicies_1]
dataset_3_v = Meiosis_Dataset(path='../SC_figures', label='pachytene_n', target=1)
dataset_3_v.im_paths = [dataset_3_v.im_paths[index] for index in indicies_2]

dataset_4 = Meiosis_Dataset(path='../SC_figures', label='leptotene', target=2)
indicies_1, indicies_2 = random_sample(len(dataset_4), .9)
dataset_4_t = Meiosis_Dataset(path='../SC_figures', label='leptotene', target=2)
dataset_4_t.im_paths = [dataset_4_t.im_paths[index] for index in indicies_1]
dataset_4_v = Meiosis_Dataset(path='../SC_figures', label='leptotene', target=2)
dataset_4_v.im_paths = [dataset_4_v.im_paths[index] for index in indicies_2]

dataset_t = ConcatDataset([dataset_1_t, 
                           dataset_2_t, 
                           dataset_3_t, 
                           dataset_4_t])
train_loader = DataLoader(dataset_t, batch_size=64, shuffle=True)

dataset_v = ConcatDataset([dataset_1_v, 
                           dataset_2_v, 
                           dataset_3_v, 
                           dataset_4_v])
val_loader = DataLoader(dataset_v, batch_size=8)

In [4]:
print(len(dataset_1), len(dataset_2), len(dataset_3), len(dataset_4), len(dataset_t))
weights = weight_balance([len(dataset_1),
                          len(dataset_2),
                          len(dataset_3),
                          len(dataset_4)])
weights = torch.FloatTensor(weights)
weights[3] = weights[3] / 4.
print(weights)

487 377 627 46 1366

 0.7890
 1.0192
 0.6128
 2.0883
[torch.FloatTensor of size 4]



In [5]:
state_dict = torch.load('./models/period_classification_4_with_ab')

In [6]:
model = Net_Period_Classification(4)
model.cuda()

Net_Period_Classification(
  (conv1a): conv_bn(
    (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv1b): conv_bn(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv2): inception_v2(
    (conv3a): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv3b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5a): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv5b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5c): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv7a): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv7b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv7c): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddi

In [7]:
'''
model.conv1a.load_state_dict(state_dict, False)
model.conv1b.load_state_dict(state_dict, False)
model.conv2.load_state_dict(state_dict, False)
model.conv3.load_state_dict(state_dict, False)
model.conv4.load_state_dict(state_dict, False)
model.conv5.load_state_dict(state_dict, False)
para_to_update = chain(model.mlp1.parameters(), model.mlp2.parameters(), model.mlp3.parameters(), model.mlp4.parameters(), model.mlp5.parameters())
'''

model.load_state_dict(state_dict)

In [8]:
para_to_update = model.parameters()
optimizer = optim.Adam(para_to_update, lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

In [9]:
loss_func = F.cross_entropy

In [10]:
def train(epoch):
    model.train()
    train_loss = 0
    acc = 0
    for batch_idx, (im, target) in enumerate(train_loader):
        input, target, weight = Variable(im).cuda(0), Variable(target.squeeze_()).cuda(0), Variable(weights).cuda(0)
        optimizer.zero_grad()
        output, _ = model(input)
        loss = loss_func(output, target, weight)
        loss.backward()
        acc += accuracy(output, target).data[0]
        train_loss += loss.data[0]
        optimizer.step()
        
    print('====> Epoch: {}t Average loss: {:.4f} Average acc: {:.3f}'.format(
          epoch, train_loss / len(train_loader), acc / len(train_loader)))

In [11]:
from torchvision.utils import save_image
from torch.nn.functional import threshold
def test(epoch):
    model.eval()
    test_loss = 0
    acc = 0
    for i, (im, target) in enumerate(val_loader):
        input, target, weight = Variable(im).cuda(0), Variable(target.squeeze_()).cuda(0), Variable(weights).cuda(0)
        output, _ = model(input)
        loss = loss_func(output, target, weight)
        acc += accuracy(output, target).data[0]
        test_loss += loss.data[0]
        
    print('====> Epoch: {}v Average loss: {:.4f} Average acc: {:.3f}'.format(
          epoch, test_loss / len(val_loader), acc / len(val_loader)))

In [ ]:
for epoch in range(101):
    train(epoch)
    test(epoch)
    

====> Epoch: 0t Average loss: 0.4545 Average acc: 0.794


/usr/local/anaconda3/lib/python3.6/site-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


====> Epoch: 0v Average loss: 0.5124 Average acc: 0.775
====> Epoch: 1t Average loss: 0.4636 Average acc: 0.769
====> Epoch: 1v Average loss: 0.4884 Average acc: 0.799
====> Epoch: 2t Average loss: 0.4616 Average acc: 0.776
====> Epoch: 2v Average loss: 0.5769 Average acc: 0.758
====> Epoch: 3t Average loss: 0.4784 Average acc: 0.776
====> Epoch: 3v Average loss: 0.4813 Average acc: 0.803
====> Epoch: 4t Average loss: 0.4500 Average acc: 0.790
====> Epoch: 4v Average loss: 0.5623 Average acc: 0.775
====> Epoch: 5t Average loss: 0.4393 Average acc: 0.789
====> Epoch: 5v Average loss: 0.5354 Average acc: 0.792
====> Epoch: 6t Average loss: 0.4577 Average acc: 0.778
====> Epoch: 6v Average loss: 0.5432 Average acc: 0.794
====> Epoch: 7t Average loss: 0.4461 Average acc: 0.799
====> Epoch: 7v Average loss: 0.5532 Average acc: 0.775
====> Epoch: 8t Average loss: 0.4320 Average acc: 0.787
====> Epoch: 8v Average loss: 0.5348 Average acc: 0.803
====> Epoch: 9t Average loss: 0.4331 Average acc

In [13]:
torch.save(model.state_dict(), './models/period_classification_4_with_ab')